In [ ]:
# import modules and define functions
from pyscf import gto, scf
import pandas as pd
import numpy as np
import os
from functools import reduce
os.environ['OMP_NUM_THREADS'] = "1" 

In [ ]:
# build H2 with minimal basis
atom = 'H 0 0 0; H 0 0 0.74'
basis = 'sto-3g'
h2_0_1 = gto.Mole()
h2_0_1.build(atom=atom,basis=basis,charge=0,spin=0)

In [ ]:
# get Hcore and nuclear_nuclear_repulsion energy
# note that these values are defined as soon as 
#   a molecule with given coordinates and basis sets is provided
h = h2_0_1.intor_symmetric('int1e_kin') + h2_0_1.intor_symmetric('int1e_nuc')
e_nuc = h2_0_1.energy_nuc()

In [ ]:
# SCF calculations
# HF
h2_0_1_rhf = scf.RHF(h2_0_1).run()

In [ ]:
# get the mo_coeffs and the 2e and fock matrix elements
mo_coeff = h2_0_1_rhf.mo_coeff
mo_occ = h2_0_1_rhf.mo_occ
veff = h2_0_1_rhf.get_veff()
f = h2_0_1_rhf.get_fock()

In [ ]:
# get H and G
H = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, h, mo_coeff])
V = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, veff, mo_coeff])
F = reduce(lambda a,b: np.dot(a,b), [mo_coeff.T, f, mo_coeff])

In [ ]:
# get orbital energies
# V = 2J - K
E_orb = H + V
E_orb = ['MO%s:%.4f Eh' %(i, e) for i, e in enumerate(np.diagonal(E_orb))]
print('Orbital energies are %s' %('. '.join(E_orb)))

In [ ]:
# get total electronic energy
H1e = np.sum(np.diagonal(H)[mo_occ>0])
J2e = np.sum(np.diagonal(V)[mo_occ>0])
E_ele = H1e*2+J2e
print('Total electronic energy is %.4f Eh' %(E_ele))

In [ ]:
# get total electronic energy
# add nuclear nuclear repulsion to the electronic energy
E_tot = E_ele + e_nuc
print('Total energy is %.4f Eh' %(E_tot))

In [ ]:
h2_0_1_rhf.analyze(verbose=5)